In [1]:
import os
import random
import pandas as pd
from typing import TypedDict, List, Literal
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver
from dotenv import load_dotenv
import re

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Configuración inicial
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Generar df con escenarios medievales
df_scenarios = pd.DataFrame({
    "scenario": ["Un joven noble se encuentra en una ciudad medieval", 
                 "Un aventurero se encuentra en una posada medieval", 
                 "Un aprendiz de mago se encuentra en una torre mágica"]
})

class AgentState(TypedDict):
    escenario: str
    heroe: str
    caramelo: str
    cantidad_puzzles: int
    max_puzzles: int
    puzzle: str
    respuesta: str
    puzzle_resuelto: bool
    story: List[str]

GENERADOR_INTRO_PROMPT = """
Eres un narrador creativo. Crea una introducción cautivadora para una historia que incorpore:
1. Nombre del héroe: {hero_name}
2. Dulce favorito del héroe: {favorite_candy}
3. Escenario aleatorio: {random_scenario}

Tu introducción debe tener 3-5 oraciones, presentar al héroe, incorporar el dulce y establecer el escenario. 
Usa un lenguaje sencillo y amigable, apropiado para niños. Puedes usar emojis para hacer la historia más divertida.
"""

PROMPT_MAGO_BLANCO = """
Eres un sabio mago que crea acertijos. Crea un acertijo sencillo o una pregunta matemática básica basada en esta historia del héroe:
{hero_story}

Tu acertijo puede ser una adivinanza simple o una pregunta matemática fácil (suma, resta, multiplicación o división básica).
Usa emojis para hacerlo más atractivo.

Para problemas matemáticos, usa la herramienta de cálculo escribiendo [CALCULAR] seguido de la expresión entre corchetes.

Presenta tu acertijo en este formato:
<acertijo>
Descripción del Acertijo: (Explica el desafío)
</acertijo>

<solucion>
Solución: (Proporciona la respuesta correcta)
</solucion>
"""

def intro_node(state: AgentState) -> AgentState:
    nombre_heroe = input("¿Cuál es el nombre de tu héroe? ")
    caramelo_favorito = input("¿Cuál es tu dulce favorito? ")
    random_scenario = df_scenarios["scenario"].sample().iloc[0]
    
    state['heroe'] = nombre_heroe
    state['caramelo'] = caramelo_favorito
    state['escenario'] = random_scenario
    
    formatted_prompt = GENERADOR_INTRO_PROMPT.format(
        hero_name=nombre_heroe,
        favorite_candy=caramelo_favorito,
        random_scenario=random_scenario
    )
    
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Genera la introducción de la historia.")
    ]
    response = model.invoke(messages)
    state['story'] = [response.content]
    print("\nHistoria generada:")
    print(state['story'][0])
    return state

def calcular(expresion):
    try:
        return eval(expresion)
    except:
        return "Error en el cálculo"

import time

def mago_blanco_node(state: AgentState) -> AgentState:
    print("\nGenerando acertijo...")
    messages = [
        SystemMessage(content=PROMPT_MAGO_BLANCO.format(hero_story=state['story'][-1])),
        HumanMessage(content="Crea un acertijo para este héroe.")
    ]
    
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = model.invoke(messages)
            print("Respuesta del modelo recibida.")
            
            # Procesar cálculos
            while '[CALCULAR]' in response.content:
                calc_match = re.search(r'\[CALCULAR\]\s*\[(.*?)\]', response.content)
                if calc_match:
                    expresion = calc_match.group(1)
                    resultado = calcular(expresion)
                    response.content = response.content.replace(calc_match.group(0), str(resultado))
            
            # Separar el acertijo de la solución
            acertijo_match = re.search(r'<acertijo>(.*?)</acertijo>', response.content, re.DOTALL)
            solucion_match = re.search(r'<solucion>(.*?)</solucion>', response.content, re.DOTALL)
            
            if acertijo_match and solucion_match:
                state['puzzle'] = acertijo_match.group(1).strip()
                state['puzzle_solution'] = solucion_match.group(1).strip()
                print("\nAcertijo del Mago Blanco:")
                print(state['puzzle'])
                break
            else:
                print(f"\nIntento {attempt + 1}: No se pudo generar un acertijo válido. Reintentando...")
                time.sleep(2)  # Espera 2 segundos antes de reintentar
        except Exception as e:
            print(f"\nError en el intento {attempt + 1}: {str(e)}")
            if attempt < max_retries - 1:
                print("Reintentando en 5 segundos...")
                time.sleep(5)
            else:
                print("Se alcanzó el máximo número de intentos. No se pudo generar un acertijo.")
                state['puzzle'] = "Error: No se pudo generar un acertijo válido."
                state['puzzle_solution'] = "N/A"
    
    state['cantidad_puzzles'] = state.get('cantidad_puzzles', 0) + 1
    return state

def human_response_node(state: AgentState) -> AgentState:
    state['respuesta'] = input("\n¿Cuál es tu respuesta al acertijo? ")
    return state

def puzzle_evaluator_node(state: AgentState) -> AgentState:
    puzzle_solution = state.get('puzzle_solution', 'N/A')
    messages = [
        SystemMessage(content="Evalúa si la respuesta del humano al acertijo es correcta. Responde solo 'True' o 'False'."),
        HumanMessage(content=f"Solución del acertijo: {puzzle_solution}\nRespuesta del humano: {state['respuesta']}\n¿Es correcta?")
    ]
    response = model.invoke(messages)
    state['puzzle_resuelto'] = response.content.strip().lower() == 'true'
    print(f"\nRespuesta {'correcta' if state['puzzle_resuelto'] else 'incorrecta'}.")
    return state

def should_continue(state: AgentState) -> Literal["continuar", "terminar"]:
    if state['puzzle'] == "Error: No se pudo generar un acertijo válido.":
        print("\n❌ Lo siento, hubo un problema al generar el acertijo. La aventura termina aquí.")
        return "terminar"
    elif state.get('puzzle_resuelto', False):
        print("\n🎉 ¡Felicidades! Has resuelto el acertijo.")
        return "terminar"
    elif state['cantidad_puzzles'] < state['max_puzzles']:
        print("\n🔄 Inténtalo de nuevo.")
        return "continuar"
    else:
        print("\n⏰ Has alcanzado el máximo número de intentos.")
        return "terminar"

# Creación del grafo
builder = StateGraph(AgentState)

# Agregar nodos
builder.add_node("presentacion_historia", intro_node)
builder.add_node("mago_blanco", mago_blanco_node)
builder.add_node("respuesta_humano", human_response_node)
builder.add_node("evaluador_puzzle", puzzle_evaluator_node)

# Definir bordes
builder.add_edge("presentacion_historia", "mago_blanco")
builder.add_edge("mago_blanco", "respuesta_humano")
builder.add_edge("respuesta_humano", "evaluador_puzzle")

# Borde condicional basado en la evaluación del acertijo
builder.add_conditional_edges(
    "evaluador_puzzle",
    should_continue,
    {
        "continuar": "mago_blanco",
        "terminar": END
    }
)

# Establecer el punto de entrada
builder.set_entry_point("presentacion_historia")

# Compilar el grafo
graph = builder.compile()

# Ejecutar el grafo
state = {
    "max_puzzles": 3,
    "cantidad_puzzles": 0,
    "story": [],
    "puzzle_solution": "N/A"
}

print("\n🧙‍♂️ ¡Bienvenido a la aventura del Mago Blanco! 🧙‍♂️")
try:
    final_state = graph.invoke(state)
    print("\n📜 Resumen final:")
    print(f"Historia: {final_state['story'][0]}")
    print(f"Acertijos resueltos: {final_state['cantidad_puzzles']}")
    if final_state.get('puzzle_resuelto', False):
        print("🎊 El héroe continúa su aventura.")
    else:
        print("🔁 El héroe debe volver a intentarlo.")
except Exception as e:
    print(f"\n❌ Error durante la ejecución del grafo: {str(e)}")
    print("La aventura no pudo completarse debido a un error inesperado.")
    
    





🧙‍♂️ ¡Bienvenido a la aventura del Mago Blanco! 🧙‍♂️

Historia generada:
¡Imagina un mundo lleno de magia y aventuras! En este mundo vive nuestro valiente héroe llamado Lube, un aprendiz de mago con un corazón tan dulce como su dulce favorito, el Rojito. Un día, mientras exploraba una misteriosa torre mágica, Lube descubrió un secreto que cambiaría su destino para siempre. ¿Qué emocionantes desafíos le esperarán en esta torre llena de sorpresas? ¡Acompáñalo en esta increíble aventura llena de magia y dulzura! 🧙🍬🏰

Generando acertijo...
Respuesta del modelo recibida.

Acertijo del Mago Blanco:
Descripción del Acertijo: 
Lube se encuentra en la torre mágica y se encuentra con tres puertas encantadas. Sobre cada puerta hay una inscripción:
- Puerta 1: "La suma de mis dígitos es 7."
- Puerta 2: "Soy el doble de la Puerta 1."
- Puerta 3: "Mi número es impar."

¿Por cuál puerta debería pasar Lube para continuar su aventura?

Respuesta incorrecta.

🔄 Inténtalo de nuevo.

Generando acertijo..